# dependencies

In [1]:
# general
import requests 
import os
import re 

# data
import pandas as pd

# local dependencies
import dirtools
from nstools import NationStatesAPI
from issue_results_api import IssuePredictionAPI

In [2]:
def pipe(nationName = 'north_kalandia', 
         headers = {'User-Agent':'Nation Info Getter', "X-Password": "koraxhun123"},
         urlStats="https://www.nationstates.net/cgi-bin/api.cgi?nation=north_kalandia&q=census;scale=all;mode=score"
):
    # get issues

    baseDir = os.getcwd()
    dirs = dirtools.Dirs(baseDir, nationName)
    nationStatesClass = NationStatesAPI(dirs, headers)
    urlIssues = nationStatesClass.generate_issues_url()
    issuesList = nationStatesClass.get_nation_issues(dirs.issuesDir, urlIssues)

    ipa = IssuePredictionAPI(dirs)
    issuePredictions = {}
    for issue in issuesList:
        ipa.get_predictions(issue)
        issuePredictions[issue] = ipa.process_predictions()
        
    
    return issuePredictions

In [3]:
issuePredictions = pipe()

In [4]:
issuePredictions

{'1101': [{'result summary': '1. a rolling news ticker tape of disclaimers runs across the bottom of the screen on all TV shows</td>',
   'Civil Rights': [-1.67, -0.0, -0.3775],
   'Economy': [-2.23, 1.0, -0.3454],
   'Wealth Gaps': [-14.02, -0.0, -0.1629],
   'Death Rate': [-7.64, 0.65, -0.1038],
   'Eco-Friendliness': [-111.11, 16.54, -2.7029],
   'Social Conservatism': [-0.44, 1.25, 0.3693],
   'Nudity': [-4.5, 0.25, -0.3262],
   'Industry: Automobile Manufacturing': [-193.88, 48.48, -5.9528],
   'Industry: Cheese Exports': [-147.6, 34.11, -5.9486],
   'Industry: Basket Weaving': [-158.47, 37.78, -7.7291],
   'Industry: Information Technology': [-236.7, 153.19, -11.9858],
   'Industry: Pizza Delivery': [-179.02, 43.83, -5.5876],
   'Industry: Trout Fishing': [-366.36, 42.57, -11.0776],
   'Industry: Arms Manufacturing': [-210.36, 162.66, -9.4403],
   'Sector: Agriculture': [-172.58, 54.92, -6.1001],
   'Industry: Beverage Sales': [-124.28, 46.21, -3.9287],
   'Industry: Timber Woodc

In [5]:
for issue in issuePredictions:
    current = issuePredictions[issue]
    print(issue, ': ', end='')
    for i in range(len(current)):
        
        choice = current[i]
        del choice['result summary']
        exec(f"df_{issue}_{i} = pd.DataFrame(choice)")
    print(i)

1101 : 4
619 : 4
1312 : 3


In [38]:
df_619_4

,Death Rate,Eco-Friendliness,Industry: Automobile Manufacturing,Industry: Cheese Exports,Industry: Basket Weaving,Industry: Information Technology,Industry: Pizza Delivery,Industry: Trout Fishing,Industry: Arms Manufacturing,Sector: Agriculture,...,Charmlessness,Human Development Index,Inclusiveness,Average Income,Average Income of Poor,Average Income of Rich,Public Education,Crime,Foreign Aid,Average Disposable Income
0,-2.0400,-37.060,-24.5000,-34.4100,-25.0700,-182.2100,-30.1300,-62.6000,-142.5600,-44.8400,...,-22.0100,-0.0200,0.0000,-833.090,-382.8400,-1791.0000,-51.4100,-1.3100,-15.4600,-874.8900
1,0.4200,1.840,11.6900,5.9400,17.6800,64.1400,9.5100,5.5700,100.9100,16.8000,...,-0.0000,0.1800,6.5800,405.710,228.0800,1557.0000,15.6400,-0.0000,0.1900,76.1000
2,-0.0298,-1.675,-1.4631,-1.8334,-2.3666,-6.2816,-1.5982,-3.2974,-4.2625,-2.1366,...,-0.6027,0.0034,2.4475,36.875,24.5271,59.2941,-3.7915,-0.1256,-0.7245,-126.5511


# get nation stats

In [7]:
nationName = 'north_kalandia'
urlStats="https://www.nationstates.net/cgi-bin/api.cgi?nation=north_kalandia&q=census;scale=all;mode=score"
headers = {'User-Agent':'Nation Info Getter', "X-Password": "koraxhun123"}
baseDir = os.getcwd()
dirs = dirtools.Dirs(baseDir, nationName)
nationStatesClass = NationStatesAPI(dirs, headers)
stats = nationStatesClass.get_nation_stats(url=urlStats, headers=headers)
with open('censusids.txt') as f:
     censusids = [line.rstrip() for line in f]
for c in df_1101_1.columns:
    if c not in censusids:
        print(c)
        raise Exception(f"{c} not in censusids")
metricStats = {}
for k in stats:
    metricStats[censusids[int(k)]]= stats[k]

# analysis

In [25]:
proc_df_1101_1['Industry: Trout Fishing']

0    16.969274
1   -49.243017
2     0.310140
Name: Industry: Trout Fishing, dtype: float64

In [26]:
metricStats['Industry: Trout Fishing']

-7.16

In [27]:
proc_df_1101_1['Industry: Trout Fishing']/metricStats['Industry: Trout Fishing']

0   -2.370010
1    6.877516
2   -0.043316
Name: Industry: Trout Fishing, dtype: float64

In [16]:
proc_df_1101_1 = df_1101_1.copy()
for c in proc_df_1101_1.columns:
    metricStats[c]
    proc_df_1101_1[c]
    proc_df_1101_1[c] = proc_df_1101_1[c]/metricStats[c]

In [17]:
proc_df_1101_1

,Civil Rights,Economy,Wealth Gaps,Death Rate,Eco-Friendliness,Social Conservatism,Nudity,Industry: Automobile Manufacturing,Industry: Cheese Exports,Industry: Basket Weaving,...,Human Development Index,Primitiveness,Scientific Advancement,Average Income,Average Income of Poor,Average Income of Rich,Public Education,Crime,Foreign Aid,Average Disposable Income
0,0.000000,-0.004947,0.000000,-0.036913,9.275547,-0.031250,0.000000,6.624138,-0.020150,-0.572700,...,-0.019802,-0.000000,-0.064511,-0.004414,-0.020324,-0.015174,-0.013217,0.000000,2.004184,-0.012344
1,0.019902,0.018360,6.075908,0.213806,-7.751825,-0.000000,0.011421,-4.569458,0.045556,0.805766,...,0.004620,-0.064644,-0.000000,0.010752,0.010326,0.054580,0.018514,0.195531,-2.193066,0.034647
2,0.004310,0.001938,0.140297,0.006889,0.166588,-0.006648,0.000974,0.346010,-0.001616,-0.042646,...,-0.001527,-0.004942,-0.004168,0.002375,-0.001084,0.010697,-0.000679,0.011173,0.010956,0.010160


In [18]:
proc_df_1101_1['Wealth Gaps']

0    0.000000
1    6.075908
2    0.140297
Name: Wealth Gaps, dtype: float64

In [19]:
final = proc_df_1101_1.loc[1,:]

In [20]:
final.sort_values()*100

Industry: Trout Fishing           -4924.301676
Industry: Gambling                -2493.130875
Industry: Insurance               -1065.066512
Industry: Furniture Restoration    -997.375566
Eco-Friendliness                   -775.182482
                                      ...     
Industry: Basket Weaving             80.576560
Weaponization                        87.500000
Industry: Timber Woodchipping       103.368618
Ignorance                           252.336449
Wealth Gaps                         607.590759
Name: 1, Length: 62, dtype: float64